In [1]:
import nltk
import requests
import json

In [108]:
text=input('Enter your input:')

Enter your input:that's what i like hello yellow submarine macarena smells like teen spirit


# NLTK part

In [102]:
tokens = nltk.word_tokenize(text)
tokens

['that', "'s", 'what', 'i', 'like', 'hollaback', 'girl', 'hello']

In [105]:
def tokenator(text):
    clean_tokens =[]
    punctuation = [",", ":", ";", ".", '"', "’", "?", "/", "-", "+", "&", "(", ")"] #"'" has been excluded
    tokens = text.split(' ') 
    for token in tokens:
        token = token.lower()
        for punc in punctuation:
            token = token.replace(punc,"")
        if len(token):
            clean_tokens.append(token)
    return clean_tokens

In [109]:
clean_tokens = tokenator(text)

In [ ]:
for token in item:
        token = token.lower()
        for punc in punctuation:
            token = token.replace(punc, "")
        tokens.append(token)
    clean_tokenized.append(tokens)
print(clean_tokenized)

In [ ]:
print(list(nltk.bigrams(tokens))[0][0])

In [95]:
clean_tokens = [word.lower() for word in tokens]
clean_tokens

['macarena', 'hello', 'fly', 'me', 'to', 'the', 'moon', 'hotline', 'bling']

In [ ]:
songs=[]

In [ ]:
songs=['hello','smells like teen spirit','single ladies']

# API part

In [ ]:
def get_most_popular_song(song, r_json):
    index = 0;
    maxval = 0;
    for i in range(len(r_json['tracks']['items'])):
        if (song in r_json['tracks']['items'][index]['name']) & (r_json['tracks']['items'][i]['popularity'] > maxval):
            index = i
            maxval = r_json['tracks']['items'][i]['popularity']
    song = r_json['tracks']['items'][index]['name'] #Song Name
    artist = r_json['tracks']['items'][index]['artists'][0]['name'] #Artist Name
    link = r_json['tracks']['items'][index]['external_urls']['spotify'] #Track on Spotify
    return '%s by %s - %s ' %(song, artist, link)

In [ ]:
playlist = {}
no_playlist = {}
for song in songs:
    parameters = {'q': song,'type': 'track', 'market': 'US', 'limit':10}
    r = requests.get('https://api.spotify.com/v1/search', params = parameters)
    if r.status_code == 200:
        playlist[song] = get_most_popular_song(song, r.json())
    else:
        no_playlist[song]= 'Failed with status error: %d' %(r.status_code)
print(playlist)
print(no_playlist)

# End of Program, Beginning of scrap code

In [ ]:
song_q = deque(clean_tokens)
song_q

In [ ]:
song2 = song_q.popleft()
print(song2)

In [ ]:
score1 = 0
score2 = 0

In [ ]:
def api_call(song2):
    qvalue = '"%s"' %(song2)
    parameters = {'q': qvalue,'type': 'track', 'market': 'US', 'limit': 10}
    r = requests.get('https://api.spotify.com/v1/search', params = parameters)
    return r

In [ ]:
def score(song2, r_json):
    scoring = [1 for i in r_json['tracks']['items'] if (song2 in i['name'].lower())]
    return sum(scoring)

In [ ]:
r = api_call(song2)
r_json = r.json()
print(r.status_code)

In [ ]:
score(song2, r_json)

In [ ]:
song2 = song2 +" "+song_q.popleft()
song2

In [ ]:
r2 = api_call(song2)
r2_json = r2.json()
print(r2.status_code)

In [ ]:
score(song2,r2_json)

# BIGRAM method

In [37]:
clean_tokens

['macarena',
 'that',
 'what',
 'i',
 'like',
 'chunky',
 'starboy',
 'fly',
 'me',
 'to',
 'the',
 'moon']

In [85]:
def score(name, r_json):
    scoring = [1 for i in r_json['tracks']['items'] if (name in i['name'].lower())]
    return sum(scoring)

In [86]:
def call_score(name):
    qvalue = '"%s"' %(name)
    parameters = {'q': qvalue,'type': 'track', 'market': 'US', 'limit': 1}
    r = requests.get('https://api.spotify.com/v1/search', params = parameters)
    if (r.status_code == 200):
        return name, score(name, r.json()), r.json()
    else:
        print('For %s we got status code: %s' %(name,r.status_code))

In [87]:
def ngram_scoring(ngram, n):
    joined_ngram = []
    if n == 1:
        joined_ngram = ngram
    else:
        joined_ngram = [' '.join(i) for i in ngram]
    for j in joined_ngram:
        name_score = call_score(j)
        if name_score[1] == 1:
            return name_score
    return

In [132]:
def remove_ngram(clean_tokens, validated_ngrams):
    if len(validated_ngrams) ==1:
        validated_tokens = validated_ngrams
    else:
        validated_tokens = validated_ngrams.split(' ')
        left_index = clean_tokens.index(validated_tokens[0])
        right_index = clean_tokens.index(validated_tokens[len(validated_tokens)-1])
        left = clean_tokens[:left_index]
        right = clean_tokens[right_index+1:]
    return left, right

In [133]:
len(clean_tokens)

12

In [134]:
def create_playlist(cleaned_tokens, n):
    if len(cleaned_tokens) == 1:
        song = ngram_scoring(cleaned_tokens, 1)
        if song:
            track = song[2]['tracks']['items'][0]['name'] #Song Name
            artist = song[2]['tracks']['items'][0]['artists'][0]['name'] #Artist Name
            link = song[2]['tracks']['items'][0]['external_urls']['spotify'] #Track on Spotify
            print("%s=> %s by %s: %s" %(song[0],track,artist,link))
        else:
            print('We could not find this word "%s"' %(cleaned_tokens[0]))
        return
    else:
        if n==1:
            token_ngrams = cleaned_tokens
        else:
            token_ngrams = list(nltk.ngrams(cleaned_tokens, n))
        song = ngram_scoring(token_ngrams, n)
        if song:
            track = song[2]['tracks']['items'][0]['name'] #Song Name
            artist = song[2]['tracks']['items'][0]['artists'][0]['name'] #Artist Name
            link = song[2]['tracks']['items'][0]['external_urls']['spotify'] #Track on Spotify
            print("%s=> %s by %s: %s" %(song[0],track,artist,link))
            left, right = remove_ngram(cleaned_tokens, song[0])
            if len(left):
                create_playlist(left, len(left))
            if len(right):
                create_playlist(right, len(right))
            return
        elif n>1:
            create_playlist(cleaned_tokens, n-1)
            return
        else:
            print('We could not find these words: %s' %(cleaned_tokens))
        return
print(text)        
create_playlist(clean_tokens, 5)

that's what i like hello yellow submarine macarena smells like teen spirit
that's what i like=> That's What I Like by Bruno Mars: https://open.spotify.com/track/0KKkJNfGyhkQ5aFogxQAPU
smells like teen spirit=> Smells Like Teen Spirit by Nirvana: https://open.spotify.com/track/5ghIJDpPoe3CfHMGu71E6T
hello yellow=> Hello (Yellow) by Erised: https://open.spotify.com/track/51Vs3U5XYPbwzdf7ZQFQ63
submarine=> Yellow Submarine - Remastered by The Beatles: https://open.spotify.com/track/50xwQXPtfNZFKFeZ0XePWc
macarena=> Mas Macarena by Gente De Zona: https://open.spotify.com/track/6rtxhUzK6FMzMtAw2Z9uZy


# FIX REPEAT WORD ISSUE

In [ ]:
# remove ngram with score from original tokens, print result playlist song

In [ ]:
# re-evaluate with leftover tokens

In [ ]:
for i in trigram:
    print(ngram_call_score(' '.join(i)))

In [ ]:
for i in bigram:
    print(ngram_call_score(' '.join(i)))

In [ ]:
for i in clean_tokens:
    print(ngram_call_score(i))

In [ ]:
names = []
in_scores_one = []
if r.status_code == 200:
    for i in range(len(r_json['tracks']['items'])):
        names.append(r_json['tracks']['items'][i]['name'].lower())
        if song2 in names[i]:
            in_scores_one.append(1)
        else:
            in_scores_one.append(0)
    print(names)
    print(in_scores_one, sum(in_scores_one))
elif r.status_code == 502:
    song2 = song2 + " " + song_q.popleft()
else:
    print('You got this Status code: %s' %(r.status_code))

In [ ]:
r_json['tracks']['items'][0]['external_urls']['spotify'] #Song link

In [ ]:
r_json['tracks'].keys()

In [ ]:
r_json['tracks']['items'][0].keys()

In [ ]:
r_json['tracks']['items'][0]['artists'][0]['name']

In [ ]:
r_json['tracks']['items'][0]['name'] # Name of the Song

In [ ]:
names = []
for i in range(20):
    names.append(r_json['tracks']['items'][i]['name'].lower())
print(names)

In [ ]:
r_json['tracks']['items'][0]['album'].keys()

In [ ]:
r_json['tracks']['items'][0]['artists'][0]['name'] # Artist

In [ ]:
r_json['tracks']['next']

In [ ]:
'hello smells' in 'hello smells from'

In [ ]:
'smells' in 'sweet dreams'

In [ ]:
queue = deque([1,2,3,4,5])
if queue:
    print('hola')
while queue:
    print(queue.popleft())
print("c'est finite")
if queue:
    print('goodbye')

In [ ]:
' '.join(['hola'])